In [1]:
import findspark

In [2]:
findspark.init('/home/guha/spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [8]:
data = spark.read.csv('/home/guha/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',inferSchema=True,header=True)

In [9]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [15]:
output = assembler.transform(data)

In [16]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                358|
|   mean|  495.2610330519258|
| stddev|  80.12364206786427|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                142|
|   mean|  509.5321781458165|
| stddev|  76.56413767006397|
|    min| 308.52774655803336|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [23]:
lrModel = lr.fit(train_data)

In [24]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.651676944266292,39.23486289913777,0.8049507877202327,61.89644653935948] Intercept: -1069.785376470991


In [25]:
test_results = lrModel.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -8.880136110286685|
|-3.5323041032122546|
| 0.4203090066914683|
| -5.192298064538363|
| -5.579872551141193|
|-15.329456379741941|
| 17.041309540683926|
|  5.689724947797856|
|-27.054306932722625|
|-1.2982805723572142|
|  -9.81846321865936|
| -2.822802643579962|
|-10.988045170662986|
| 12.255798155197738|
|   8.39806621238938|
|  6.247340352731726|
| -8.717747595177798|
|  4.478256542770737|
| 0.8138803021388412|
|  4.328270585844052|
+-------------------+
only showing top 20 rows



In [27]:
test_results.r2

0.9834941421276027

In [28]:
test_results.rootMeanSquaredError

9.80189013129059

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[31.1280900496166...|
|[31.2681042107507...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.5257524169682...|
|[31.5741380228732...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.7216523605090...|
|[31.8279790554652...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.9262720263601...|
|[31.9549038566348...|
|[31.9764800614612...|
|[32.0085045178551...|
|[32.0215955013870...|
|[32.0444861274404...|
|[32.0478009788678...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lrModel.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.1280900496166...| 566.1328228573414|
|[31.2681042107507...| 427.0028372770362|
|[31.3895854806643...| 409.6493020532914|
|[31.4252268808548...| 535.9590167193003|
|[31.5257524169682...| 449.5454993610231|
|[31.5741380228732...| 559.7387285403288|
|[31.6098395733896...|427.50424011042423|
|[31.6548096756927...|469.57369877975066|
|[31.6739155032749...|502.77937484260383|
|[31.7216523605090...|349.07520720422986|
|[31.8279790554652...| 449.8212107656009|
|[31.8530748017465...|462.10792610593194|
|[31.8648325480987...|450.87932564747666|
|[31.9262720263601...|379.94913528912866|
|[31.9549038566348...| 431.5998137275376|
|[31.9764800614612...| 324.3471056813685|
|[32.0085045178551...| 451.9149686239332|
|[32.0215955013870...| 517.0939182150566|
|[32.0444861274404...|447.41594888441114|
|[32.0478009788678...| 509.1223006002524|
+--------------------+------------